<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Build Fast with AI](https://img.shields.io/badge/BuildFastWithAI-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://www.buildfastwithai.com/genai-course)
[![EduChain GitHub](https://img.shields.io/github/stars/satvik314/educhain?style=for-the-badge&logo=github&color=gold)](https://github.com/satvik314/educhain)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Ad-sA50hxkGlPGwoGIrO3PBoPxCvKhms#scrollTo=vBmNHxOzvMln)
## Master Generative AI in 6 Weeks
**What You'll Learn:**
- Build with Latest LLMs
- Create Custom AI Apps
- Learn from Industry Experts
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)
*Empowering the Next Generation of AI Innovators

##Phidata :- Build Agents with memory, knowledge, tools and reasoning


###What is phidata?
Phidata is a framework for building agentic systems, engineers use phidata to:

* Build Agents with memory, knowledge, tools and reasoning.
* Build teams of Agents that can work together.
* Chat with Agents using a beautiful Agent UI.
* Monitor, evaluate and optimize Agents.
* Build agentic systems i.e. applications with an API, database and vectordb.

### Setup
To get started, we need to install the last version of phidata.

In [ ]:
!pip install phidata duckduckgo-search openai yfinance httpx==0.27.2 lancedb tantivy pypdf

###Setting Up API Keys

In [2]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

###Web Search Agent :- Let's start by building a simple agent that can search the web

In [3]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.tools.duckduckgo import DuckDuckGo

web_agent = Agent(
    name="Web Agent",
    model=OpenAIChat(id="gpt-4"),
    tools=[DuckDuckGo()],
    instructions=["Always include sources"],
    show_tool_calls=True,
    markdown=True,
)

### Simple Query

In [4]:
web_agent.print_response("Whats happening in France?", stream=True)

Output()

### Creating Agents for Financial Analysis

We'll create specialized agents using Phidata - a web agent for general research and a finance agent for market data analysis. These agents will be combined into a team to provide comprehensive financial insights.


In [5]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.yfinance import YFinanceTools

web_agent = Agent(
    name="Web Agent",
    role="Search the web for information",
    model=OpenAIChat(id="gpt-4o-mini"),
    tools=[DuckDuckGo()],
    markdown=True,
    show_tool_calls=True,
)

finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data",
    model=OpenAIChat(id="gpt-4o-mini"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True, company_news=True)],
    instructions=["Always use tables to display data"],
    markdown=True,
    show_tool_calls=True,
)

###Create Agent Team

In [6]:
agent_team = Agent(
    team=[web_agent, finance_agent],
    show_tool_calls=True,
    markdown=True,
)

### Simple Query

In [7]:
agent_team.print_response("How much did Nvidia stock increase in last 1 year?", stream=True)

Output()

### Comparing Stocks

In [8]:
def compare_stocks(agent, symbol1, symbol2, period='1y'):
    query = f"Compare the performance metrics of {symbol1} and {symbol2} over the last {period}. Do a detailed analysis on P/E ratio"
    response = agent_team.print_response(
        query,
        stream=True,
        show_full_reasoning=True
    )
    return response

### Compare two stocks


In [9]:
# Compare two stocks
compare_stocks(finance_agent, "NVDA", "TSLA", "1y")

Output()

###Reasoning Agents


In [10]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat

task = (
    "Three missionaries and three cannibals need to cross a river. "
    "They have a boat that can carry up to two people at a time. "
    "If, at any time, the cannibals outnumber the missionaries on either side of the river, the cannibals will eat the missionaries. "
    "How can all six people get across the river safely? Provide a step-by-step solution and show the solutions as an ascii diagram"
)

reasoning_agent = Agent(model=OpenAIChat(id="gpt-4o"), reasoning=True, markdown=True, structured_outputs=True)
reasoning_agent.print_response(task, stream=True, show_full_reasoning=True)

Output()

###RAG Agent


Instead of always inserting the "context" into the prompt, the RAG Agent can search its knowledge base (vector db) for the specific information it needs to achieve its task.

In [17]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.embedder.openai import OpenAIEmbedder
from phi.knowledge.pdf import PDFUrlKnowledgeBase
from phi.vectordb.lancedb import LanceDb, SearchType

# Create a knowledge base from a PDF
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://phi-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    # Use LanceDB as the vector database
    vector_db=LanceDb(
        table_name="recipes",
        uri="tmp/lancedb",
        search_type=SearchType.vector,
        embedder=OpenAIEmbedder(model="text-embedding-3-small"),
    ),
)
# Comment out after first run as the knowledge base is loaded
knowledge_base.load()

agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    # Add the knowledge base to the agent
    knowledge=knowledge_base,
    show_tool_calls=True,
    markdown=True,
)
agent.print_response("How do I make chicken and galangal in coconut milk soup", stream=True)

INFO     Creating collection

INFO     Loading knowledge base

INFO     Reading: https://phi-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO     Added 14 documents to knowledge base

Output()